### IBM Data Science Capstone Project
#### The Battle of Neighborhoods

In [134]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from re import sub
from decimal import Decimal
from sklearn import linear_model
from sklearn import preprocessing

print('Libraries imported.')

Libraries imported.


In [2]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [3]:
URL = "https://www.zip-codes.com/zip-code-radius-finder.asp?zipmileslow=0&zipmileshigh=30&zip1=03869&submit=Search"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find("div", {"id" : "tableview"})
list_table = tableDataText(table)
df = pd.DataFrame(list_table[1:], columns=list_table[0])
df = df.drop('#', axis = 1)
df = df[:-1]
df.tail()

,Zip,City,County,St,Country,Distance,
98,04061,NORTH WATERBORO,YORK,ME,US,29.18 miles,
99,03810,ALTON BAY,BELKNAP,NH,US,29.38 miles,
100,04006,BIDDEFORD POOL,YORK,ME,US,29.53 miles,
101,04056,NEWFIELD,YORK,ME,US,29.56 miles,
102,04063,OCEAN PARK,YORK,ME,US,29.60 miles,


In [4]:
def getIncomeData(base_URL, num_pages):
    df_income = pd.DataFrame()
    i = 0
    while i<num_pages:
        if i == 0:
            URL = base_URL.format('')
        else:
            URL = base_URL.format(str(i+1) + ".")
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find('div', {'id' : 'ctl00_ContentPlaceHolder1_final_content'})
        income_table = tableDataText(table)
        df_income = df_income.append(pd.DataFrame(income_table[4:], columns=income_table[3]))
        i += 1
    return df_income[:-3]

In [5]:
me_income = getIncomeData("http://zipatlas.com/us/me/zip-code-comparison/median-household-income.{}htm", 4)
nh_income = getIncomeData("http://zipatlas.com/us/nh/zip-code-comparison/median-household-income.{}htm", 3)
income_data = nh_income.append(me_income)
income_data[['Latitude', 'Longitude']] = income_data['Location'].str.split(',', expand = True)
income_data = income_data.drop(['#', 'Location', 'National Rank'], axis = 1)
income_data = income_data.rename(columns = {'Avg. Income/H/hold': 'Household Income'})
income_data.head()

,Zip Code,City,Population,Household Income,Latitude,Longitude
0,03087,"Windham,New Hampshire","10,709","$94,794.00",42.811092,-71.302688
1,03049,"Hollis,New Hampshire","7,017","$92,847.00",42.749626,-71.585401
2,03031,"Amherst,New Hampshire","10,812","$88,671.00",42.874854,-71.600512
3,03750,"Etna,New Hampshire",962,"$86,421.00",43.708188,-72.195047
4,03233,"Elkins,New Hampshire",234,"$85,247.00",43.438900,-71.952889


In [6]:
df = pd.merge(df, income_data,how="inner", left_on="Zip", right_on = "Zip Code")
df = df.drop(['City_x', 'County', 'St', 'Country', 'Zip'], axis = 1)
df = df.rename(columns = {'City_y': 'City'})
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])
df.head()

,Distance,,Zip Code,City,Population,Household Income,Latitude,Longitude
0,0.00 miles,,03869,"Rollinsford,New Hampshire","2,407","$48,380.00",43.219845,-70.843112
1,3.20 miles,,03878,"Somersworth,New Hampshire","11,483","$43,177.00",43.253476,-70.887944
2,3.52 miles,,03820,"Dover,New Hampshire","28,537","$44,493.00",43.187186,-70.902839
3,5.29 miles,,03908,"South Berwick,Maine","6,682","$53,294.00",43.230492,-70.752426
4,5.83 miles,,03901,"Berwick,Maine","6,338","$44,519.00",43.293960,-70.836455


In [135]:
rollinsford = df[df['City'] == 'Rollinsford,New Hampshire']
map_rollinsford = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rollinsford)  

map_rollinsford

In [8]:
CLIENT_ID = 'W02GLHQCAYNTW4FFRCROILCNH5DPQPBQOT2RFBQCX4GNLERQ' # your Foursquare ID
CLIENT_SECRET = 'T5NZATV1DSRDNOTCQPVJTMVN5SWZRZHB4KE4MFW0PZMFPJRZ' # your Foursquare Secret
ACCESS_TOKEN = 'TTU5B5RAIMXLGMKTTL0YW2K0QBJJLKYYP2MMP0JQ3QCCGV3Y' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W02GLHQCAYNTW4FFRCROILCNH5DPQPBQOT2RFBQCX4GNLERQ
CLIENT_SECRET:T5NZATV1DSRDNOTCQPVJTMVN5SWZRZHB4KE4MFW0PZMFPJRZ


In [9]:
radius = 3000
LIMIT = 500

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                             'City Latitude', 
                             'City Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [11]:
nearby_venues = getNearbyVenues(names = df['City'],
                                latitudes = df['Latitude'],
                                longitudes = df['Longitude'],
                                radius = 3000)
nearby_venues.shape

(1483, 7)

In [12]:
venue_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix = "", prefix_sep="")
venue_onehot['City'] = nearby_venues['City']

tmp = venue_onehot.pop('City')
venue_onehot.insert(0, 'City', tmp)

city_venues = venue_onehot.groupby('City').mean().reset_index()
city_venues.head()

,City,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Veterinarian,Video Game Store,Video Store,Vineyard,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Acton,Maine",0.0,0.0,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alfred,Maine",0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Alton Bay,New Hampshire",0.0,0.0,0.000000,0.000000,0.0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Alton,New Hampshire",0.0,0.0,0.000000,0.200000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Barnstead,New Hampshire",0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print(list(city_venues.columns))

['City', 'Accessories Store', 'Airport', 'Airport Terminal', 'American Restaurant', 'Antique Shop', 'Arcade', 'Art Gallery', 'Arts & Crafts Store', 'Asian Restaurant', 'Assisted Living', 'Athletics & Sports', 'Automotive Shop', 'BBQ Joint', 'Bagel Shop', 'Bakery', 'Bank', 'Bar', 'Baseball Field', 'Basketball Court', 'Basketball Stadium', 'Beach', 'Bed & Breakfast', 'Beer Garden', 'Beer Store', 'Big Box Store', 'Bistro', 'Board Shop', 'Boat Launch', 'Boat or Ferry', 'Bookstore', 'Border Crossing', 'Boutique', 'Bowling Alley', 'Breakfast Spot', 'Brewery', 'Burger Joint', 'Bus Station', 'Business Service', 'Butcher', 'Cafeteria', 'Café', 'Campground', 'Candy Store', 'Caribbean Restaurant', 'Carpet Store', 'Chinese Restaurant', 'Chocolate Shop', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Cafeteria', 'College Library', 'Comic Shop', 'Construction & Landscaping', 'Convenience Store', 'Cosmetics Shop', 'Deli / Bodega', 'Department Store', 'Dessert Shop', 'Diner', 'Disc Golf', '

In [18]:
shopping_cols = ['Accessories Store', 'Antique Shop', 'Arts & Crafts Store', 'Big Box Store', 'Bookstore', 'Boutique', 
                 'Carpet Store', 'Clothing Store', 'Comic Shop', 'Convenience Store', 'Cosmetics Shop', 'Department Store', 
                 'Electronics Store', 'Fabric Shop', 'Flea Market', 'Flower Shop', 'Furniture / Home Store', 'Garden Center', 
                 'Gas Station', 'Gift Shop', 'Gourmet Shop', 'Hardware Store', 'Hobby Shop', 'Jewelry Store', 'Kids Store', 
                 'Liquor Store', 'Market', 'Mattress Store', "Men's Store", 'Discount Store', 'Mobile Phone Shop', 
                 'Miscellaneous Shop', 'Motorcycle Shop', 'Music Store', 'Optical Shop', 'Other Repair Shop', 'Outlet Mall', 
                 'Paper / Office Supplies Store', 'Pet Store', 'Shipping Store', 'Shoe Store', 'Shopping Mall', 'Shopping Plaza',
                 'Smoke Shop', 'Supplement Shop', 'Thrift / Vintage Store', 'Toy / Game Store', 'Warehouse Store', 
                 "Women's Store", 'Video Game Store', 'Video Store']

travel_cols = ['Airport', 'Airport Terminal', 'Rental Car Location', 'Toll Plaza', 'Tourist Information Center', 'Train Station']

dining_cols = ['American Restaurant', 'Asian Restaurant', 'Bagel Shop', 'Bakery', 'BBQ Joint', 'Bar', 'Bistro','Breakfast Spot',
               'Brewery', 'Burger Joint', 'Butcher', 'Cafeteria', 'Café','Candy Store','Caribbean Restaurant', 'Chinese Restaurant',
               'Cocktail Bar', 'Coffee Shop', 'College Cafeteria', 'Deli / Bodega', 'Dessert Shop', 'Diner', 'Donut Shop', 
               'Falafel Restaurant', 'Fast Food Restaurant', 'Fish Market', 'Food', 'Food & Drink Shop', 'Food Court',
               'Food Truck', 'French Restaurant', 'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Chocolate Shop', 'Farmers Market', 
               'Gastropub', 'Greek Restaurant', 'Grocery Store', 'Hawaiian Restaurant', 'Indian Restaurant', 'Italian Restaurant',
               'Japanese Restaurant', 'Juice Bar', 'Korean Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 
               'New American Restaurant', 'Pub', 'Restaurant', 'Hot Dog Joint', 'Ice Cream Shop', 'Pizza Place', 'Sandwich Place',
               'Seafood Restaurant', 'Smoothie Shop', 'Snack Place', 'Soup Place', 'Steakhouse', 'Supermarket', 
               'Sushi Restaurant', 'Taco Place', 'Tea Room', 'Tex-Mex Restaurant', 'Thai Restaurant', 
               'Vegetarian / Vegan Restaurant', 'Wine Shop', 'Wings Joint']

fitness_cols = ['Athletics & Sports', 'Baseball Field', 'Basketball Court', 'Basketball Stadium', 'Board Shop', 'Golf Course', 
                'Gym', 'Gym / Fitness Center', 'Gymnastics Gym', 'Health & Beauty Service', 'Hockey Arena', 'Skating Rink', 
                'Soccer Field', 'Soccer Stadium', 'Sporting Goods Shop', 'Weight Loss Center']

entertainment_cols = ['Arcade', 'Art Gallery', 'Beer Garden', 'Beer Store',  'Bowling Alley',
                      'Dog Run', 'College Library', 'Disc Golf', 'Dive Bar', 'Drive-in Theater', 'Event Space', 'Gay Bar', 
                      'Gun Range', 'Harbor / Marina', 'Historic Site', 'History Museum', 'Intersection', 'Lounge', 'Mini Golf', 
                      'Movie Theater', 'Museum', 'Music Venue', 'Performing Arts Venue', 'Photography Studio', 'Plaza', 
                      'Recording Studio', 'Salon / Barbershop', 'Sports Bar', 'Summer Camp', 'Theater', 'Wine Bar', 
                      'Rock Club', 'Science Museum']

outdoor_cols = ['Boat Launch', 'Boat or Ferry', 'Farm', 'Garden','Lake', 'Lighthouse', 'Mountain', 'Outdoor Sculpture', 'Park', 
                'Pier', 'Playground', 'Pool', 'River', 'Scenic Lookout', 'Sculpture Garden', 'State / Provincial Park',
                'Surf Spot', 'Trail', 'Vineyard', 'Nature Preserve']

hospitality_cols = ['Bed & Breakfast', 'Campground', 'Hotel', 'Inn', 'Motel', 'RV Park', 'Resort']

other_cols = ['Assisted Living', 'Automotive Shop', 'Bank', 'Beach', 'Border Crossing','Bus Station', 'Business Service', 
              'Construction & Landscaping', 'Home Service', 'Insurance Office', 'Laundromat', 'Lawyer', 'Massage Studio', 
              'Moving Target', 'Multiplex', 'Pharmacy', 'Post Office', 'Real Estate Office', 'Rest Area', 'Storage Facility', 
              'Veterinarian']

col_types = [shopping_cols, travel_cols, dining_cols, fitness_cols, entertainment_cols, outdoor_cols, hospitality_cols, other_cols] 

In [23]:
categorized_venues = pd.DataFrame()
categorized_venues['City'] = city_venues['City']
categorized_venues['Shopping'] = city_venues[shopping_cols].sum(axis = 1)
categorized_venues['Travel'] = city_venues[travel_cols].sum(axis = 1)
categorized_venues['Dining'] = city_venues[dining_cols].sum(axis = 1)
categorized_venues['Fitness'] = city_venues[fitness_cols].sum(axis = 1)
categorized_venues['Entertainment'] = city_venues[entertainment_cols].sum(axis = 1)
categorized_venues['Outdoor'] = city_venues[outdoor_cols].sum(axis = 1)
categorized_venues['Hospitality'] = city_venues[hospitality_cols].sum(axis = 1)
categorized_venues['Other'] = city_venues[other_cols].sum(axis = 1)
categorized_venues.head()

,City,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
0,"Acton,Maine",0.000000,0.166667,0.333333,0.0,0.166667,0.00,0.166667,0.166667
1,"Alfred,Maine",0.250000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.750000
2,"Alton Bay,New Hampshire",0.000000,0.000000,0.000000,0.0,0.100000,0.70,0.200000,0.000000
3,"Alton,New Hampshire",0.050000,0.000000,0.650000,0.0,0.150000,0.05,0.050000,0.050000
4,"Barnstead,New Hampshire",0.166667,0.000000,0.000000,0.0,0.500000,0.00,0.000000,0.333333


In [128]:
df_info = pd.merge(df[['City', 'Population', 'Household Income']], categorized_venues,how="inner", on = 'City')
df_info['Population'] = df_info['Population'].str.replace(',', '').astype(int)
df_info['Household Income'].replace(to_replace='[^0-9]+', value='',inplace=True,regex=True)
df_info['Household Income'] = df_info['Household Income'].astype(float)/100
df_info.head()

,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
0,"Rollinsford,New Hampshire",2407,48380.0,0.227273,0.000000,0.363636,0.090909,0.045455,0.181818,0.0,0.090909
1,"Somersworth,New Hampshire",11483,43177.0,0.327586,0.017241,0.500000,0.068966,0.034483,0.000000,0.0,0.051724
2,"Dover,New Hampshire",28537,44493.0,0.205882,0.000000,0.588235,0.044118,0.088235,0.029412,0.0,0.044118
3,"South Berwick,Maine",6682,53294.0,0.000000,0.000000,0.250000,0.250000,0.250000,0.250000,0.0,0.000000
4,"Berwick,Maine",6338,44519.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.500000,0.0,0.000000


In [152]:
k_clusters = 5
df_cluster = df_info.copy()
df_cluster_scaled = df_cluster.drop('City', 1)
df_cluster_scaled = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df_cluster_scaled), columns = df_cluster.columns[1:])
kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(df_cluster_scaled)
df_cluster_scaled.insert(0, 'City', df_cluster['City'])
df_cluster.insert(0, 'Cluster #', kmeans.labels_)
df_cluster.head()

,Cluster #,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
0,3,"Rollinsford,New Hampshire",2407,48380.0,0.227273,0.000000,0.363636,0.090909,0.045455,0.181818,0.0,0.090909
1,2,"Somersworth,New Hampshire",11483,43177.0,0.327586,0.017241,0.500000,0.068966,0.034483,0.000000,0.0,0.051724
2,2,"Dover,New Hampshire",28537,44493.0,0.205882,0.000000,0.588235,0.044118,0.088235,0.029412,0.0,0.044118
3,0,"South Berwick,Maine",6682,53294.0,0.000000,0.000000,0.250000,0.250000,0.250000,0.250000,0.0,0.000000
4,3,"Berwick,Maine",6338,44519.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.500000,0.0,0.000000


In [153]:
df_mapping = pd.merge(df_cluster, df[['City', 'Latitude', 'Longitude']], how = 'left', on = 'City')
df_mapping.head()

,Cluster #,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other,Latitude,Longitude
0,3,"Rollinsford,New Hampshire",2407,48380.0,0.227273,0.000000,0.363636,0.090909,0.045455,0.181818,0.0,0.090909,43.219845,-70.843112
1,2,"Somersworth,New Hampshire",11483,43177.0,0.327586,0.017241,0.500000,0.068966,0.034483,0.000000,0.0,0.051724,43.253476,-70.887944
2,2,"Dover,New Hampshire",28537,44493.0,0.205882,0.000000,0.588235,0.044118,0.088235,0.029412,0.0,0.044118,43.187186,-70.902839
3,0,"South Berwick,Maine",6682,53294.0,0.000000,0.000000,0.250000,0.250000,0.250000,0.250000,0.0,0.000000,43.230492,-70.752426
4,3,"Berwick,Maine",6338,44519.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.500000,0.0,0.000000,43.293960,-70.836455


In [154]:
# create map
map_clusters = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_mapping['Latitude'], df_mapping['Longitude'], df_mapping['City'], df_mapping['Cluster #']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [155]:
df_cluster.groupby(['Cluster #']).mean()

,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
Cluster #,,,,,,,,,,
0,4640.750000,61020.666667,0.122854,0.003391,0.318736,0.177808,0.030009,0.179436,0.027238,0.140528
1,4140.750000,44076.000000,0.145833,0.083333,0.000000,0.000000,0.000000,0.166667,0.000000,0.604167
2,19267.500000,44965.100000,0.181935,0.025028,0.488863,0.084890,0.038729,0.066324,0.017052,0.097179
3,4552.517241,48206.103448,0.217879,0.005435,0.484384,0.023002,0.038615,0.057434,0.026730,0.146522
4,2614.333333,50915.444444,0.062169,0.050265,0.123423,0.000000,0.250156,0.316823,0.084392,0.112773


In [160]:
df_cluster0 = df_cluster[df_cluster['Cluster #'] == 0]
df_cluster0.sort_values(by='Fitness')

,Cluster #,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
19,0,"New Castle,New Hampshire",1011,83708.0,0.000000,0.000000,0.411765,0.000000,0.176471,0.294118,0.000000,0.117647
63,0,"Sandown,New Hampshire",5205,67209.0,0.000000,0.000000,0.375000,0.000000,0.000000,0.250000,0.000000,0.375000
39,0,"Hampton Falls,New Hampshire",1891,75513.0,0.400000,0.066667,0.133333,0.066667,0.066667,0.066667,0.066667,0.133333
54,0,"Kingston,New Hampshire",5852,61522.0,0.133333,0.000000,0.400000,0.066667,0.000000,0.200000,0.066667,0.133333
27,0,"Rye,New Hampshire",5182,63152.0,0.071429,0.000000,0.428571,0.071429,0.000000,0.178571,0.000000,0.250000
26,0,"Stratham,New Hampshire",6355,76726.0,0.257143,0.000000,0.485714,0.085714,0.057143,0.028571,0.000000,0.085714
49,0,"Raymond,New Hampshire",9666,48829.0,0.363636,0.000000,0.181818,0.090909,0.000000,0.181818,0.000000,0.181818
70,0,"Hampstead,New Hampshire",6276,70536.0,0.074074,0.000000,0.444444,0.111111,0.111111,0.037037,0.037037,0.185185
73,0,"Plaistow,New Hampshire",7747,61707.0,0.382353,0.014706,0.338235,0.117647,0.058824,0.014706,0.000000,0.073529
25,0,"Newfields,New Hampshire",1551,71375.0,0.000000,0.000000,0.250000,0.125000,0.000000,0.500000,0.000000,0.125000
